In [1]:
import numpy as np
from tensorflow.keras.applications import VGG19
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from skimage.color import rgb2gray
from skimage.feature import hog
from skimage.io import imread
import os
import pandas as pd
from tensorflow.keras.preprocessing.image import img_to_array, load_img
import cv2
import matplotlib.pyplot as plt
import random
import joblib
import time

2024-06-14 02:37:57.043700: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-14 02:37:57.049071: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-14 02:37:57.131403: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-14 02:37:58.892750: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
# Списки с названиями классов и их индексами
list_train = ['Melanoma Skin Cancer Nevi and Moles',
              'Acne and Rosacea Photos',
              'Atopic Dermatitis Photos',
              'Nail Fungus and other Nail Disease',
              'Seborrheic Keratoses and other Benign Tumors',
              'Psoriasis pictures Lichen Planus and related diseases']

class_to_idx = {cls_name: idx for idx, cls_name in enumerate(list_train)}

In [3]:
# Функция для создания словаря с путями и метками
def create_image_dict(image_dir, class_to_idx):
    image_dict = {"image_path": [], "target": []}
    for cls_name, cls_idx in class_to_idx.items():
        class_dir = os.path.join(image_dir, cls_name)
        if os.path.isdir(class_dir):
            for img_name in os.listdir(class_dir):
                img_path = os.path.join(class_dir, img_name)
                image_dict["image_path"].append(img_path)
                image_dict["target"].append(cls_idx)
    return image_dict

In [4]:
# Создание словарей для обучающих и тестовых данных
train_dir = "dermnet/train"
test_dir = "dermnet/test"

train_dictionary = create_image_dict(train_dir, class_to_idx)
test_dictionary = create_image_dict(test_dir, class_to_idx)
print(len(train_dictionary['image_path']), len(test_dictionary['image_path']))

3885 1507


In [ ]:
def add_random_noise(image):
    row, col, ch = image.shape
    mean = 0
    var = 0.1
    sigma = var ** 0.5
    gauss = np.random.normal(mean, sigma, (row, col, ch))
    gauss = gauss.reshape(row, col, ch)
    noisy = image + gauss
    return np.clip(noisy, 0, 255).astype(np.uint8)

def rotate_image(image, angle):
    (h, w) = image.shape[:2]
    center = (w // 2, h // 2)
    M = cv2.getRotationMatrix2D(center, angle, 1.0)
    rotated = cv2.warpAffine(image, M, (w, h))
    return rotated

In [ ]:
def augment_images(images, targets):
    augmented_images = []
    augmented_targets = []

    for img, target in zip(images, targets):
        # Original image
        augmented_images.append(img)
        augmented_targets.append(target)
        
        # Rotate image by a random angle between -45 and 45 degrees
        angle = random.uniform(-45, 45)
        rotated_img = rotate_image(img, angle)
        augmented_images.append(rotated_img)
        augmented_targets.append(target)
        
        # Add random noise to image
        # noisy_img = add_random_noise(img)
        # augmented_images.append(noisy_img)
        # augmented_targets.append(target)
        
    return augmented_images, augmented_targets

In [5]:
# Извлечение изображений и меток из train_dictionary
train_images = [imread(path) for path in train_dictionary["image_path"]]
# train_labels = np.array(train_dictionary["target"])

# augmented_images, augmented_labels = augment_images(train_images, train_labels)

In [6]:
test_images = [imread(path) for path in test_dictionary["image_path"]]

In [7]:
def preprocess(img):
    desired_size = (100, 100)
    img = cv2.resize(img, desired_size)
    alpha = 1  # коэффициент яркости
    beta = 3   # коэффициент контрастности

    matrix = np.array([[1.5, 0.0, 0.0],
                       [0.0, 1.08, 0.0],
                       [0.0, 0.0, 0.8]])
    img = cv2.transform(img, matrix)

    img = cv2.convertScaleAbs(img, alpha=alpha, beta=beta)
    return img

In [8]:
tr_images = []
ts_images = []
for img in train_images:
    tr_images.append(preprocess(img))

In [9]:
for img in test_images:
    ts_images.append(preprocess(img))

In [ ]:
def extract_hog_features(image):
    # image = imread(image_path)
    gray_image = rgb2gray(image)
    features, hog_image = hog(gray_image,
                      orientations=9,
                      pixels_per_cell=(8, 8),
                      cells_per_block=(2, 2),
                      block_norm='L2-Hys',
                      visualize=True)
    return features, hog_image

In [ ]:
random_index = random.randint(0, len(tr_images) - 1)
image = tr_images[random_index]
sample_features, hog_image = extract_hog_features(image)

# Визуализация изображения HOG
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6), sharex=True, sharey=True)

ax1.imshow(image)
ax1.set_title('Исходное изображение')

ax2.imshow(hog_image, cmap=plt.cm.gray)
ax2.set_title('Изображение HOG')
plt.show()

In [ ]:
def extract_surf_features(images):
    surf = cv2.SURF_create()
    all_descriptors = []
    image_indices = []

    for idx, image in enumerate(images):
        gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        keypoints, descriptors = surf.detectAndCompute(gray_image, None)
        if descriptors is not None:
            all_descriptors.append(descriptors)
            image_indices.extend([idx] * descriptors.shape[0])

    return np.vstack(all_descriptors), np.array(image_indices)

In [ ]:
mean_surf_features = []
for i in range(200):
    random_index = random.randint(0, len(tr_images) - 1)
    image = tr_images[random_index]
    descriptors, image_indices = extract_surf_features([image])
    mean_surf_features.append(descriptors.shape[0])
    print(f"Количество признаков, выделенных из изображения: {descriptors.shape[0]}")
print(f'Mean surf features count: {mean_surf_features.mean}')

In [10]:
def flatten_features(images):
    flattened_features = []
    for img in images:
        flattened_img = img.flatten()  # Простое выпрямление изображения в одномерный массив
        flattened_features.append(flattened_img)
    return np.array(flattened_features)

In [ ]:
train_features = [extract_hog_features(img) for img in tr_images]
# train_features = [extract_hog_features(path) for path in train_dictionary["image_path"]]
train_features = np.array(train_features)
train_labels = np.array(train_dictionary["target"])

# train_features = np.array(train_features)
# train_labels = np.array(train_labels)

# Извлечение изображений и меток из test_dictionary
test_features = [extract_hog_features(img) for img in ts_images]
test_features = np.array(test_features)
test_labels = np.array(test_dictionary["target"])

In [ ]:
train_features, train_image_indices = extract_surf_features(tr_images)
test_features, _ = extract_surf_features(ts_images)

# Приведение меток в соответствие с индексами признаков
train_labels = np.array(train_dictionary["target"])[train_image_indices]
test_labels = np.array(test_dictionary["target"])

In [11]:
train_features = flatten_features(tr_images)
test_features = flatten_features(ts_images)

train_labels = np.array(train_dictionary["target"])
test_labels = np.array(test_dictionary["target"])

In [12]:
def train_and_predict(train_features, train_labels, test_features):
    predictions = np.zeros((test_features.shape[0], len(list_train)))
    
    for class_idx, class_name in enumerate(list_train):
        # Создаем бинарные метки для текущего класса
        binary_train_labels = (train_labels == class_idx).astype(int)
        
        # Разбиваем данные на обучающую и валидационную выборки
        X_train, X_val, y_train, y_val = train_test_split(train_features, binary_train_labels, test_size=0.2, random_state=42)
        
        # Обучаем модель
        # model = LogisticRegression(max_iter=1000, random_state=42)
        # model = RandomForestClassifier(n_estimators=100, random_state=42)
        # model = GradientBoostingClassifier(n_estimators=100, random_state=42)
        model = SVC(kernel='linear', probability=True, random_state=42)
        # model = KNeighborsClassifier(n_neighbors=5)
        model.fit(X_train, y_train)
        
        # Делаем предсказания на тестовой выборке
        class_predictions = model.predict(test_features)
        predictions[:, class_idx] = class_predictions
    
    return predictions

In [13]:
start = time.time()
test_predictions = train_and_predict(train_features, train_labels, test_features)
end = time.time()
print(f'time: {end-start} sec')

time: 2709.7732362747192 sec


In [14]:
start = time.time()
train_predictions = train_and_predict(train_features, train_labels, train_features)
end = time.time()
print(f'time: {end-start} sec')

time: 2820.3386068344116 sec


In [15]:
final_predictions = np.argmax(test_predictions, axis=1)
final_predictions_train = np.argmax(train_predictions, axis=1)

# Сопоставление меток с названиями классов
predicted_classes = [list_train[pred] for pred in final_predictions]
predicted_train_classes = [list_train[pred] for pred in final_predictions_train]

In [16]:
accuracy = accuracy_score(test_labels, final_predictions)
print(f"Accuracy: {accuracy:.4f}")

train_accuracy = accuracy_score(train_labels, final_predictions_train)
print(f"Accuracy train: {train_accuracy:.4f}")

Accuracy: 0.3915
Accuracy train: 0.8839


In [17]:
for img_path, true_class, pred_class in zip(test_dictionary["image_path"], test_labels, final_predictions):
    true_class_name = list_train[true_class]
    pred_class_name = list_train[pred_class]
    print(f"True Class: {true_class_name}, Predicted Class: {pred_class_name}")

True Class: Melanoma Skin Cancer Nevi and Moles, Predicted Class: Melanoma Skin Cancer Nevi and Moles
True Class: Melanoma Skin Cancer Nevi and Moles, Predicted Class: Melanoma Skin Cancer Nevi and Moles
True Class: Melanoma Skin Cancer Nevi and Moles, Predicted Class: Melanoma Skin Cancer Nevi and Moles
True Class: Melanoma Skin Cancer Nevi and Moles, Predicted Class: Psoriasis pictures Lichen Planus and related diseases
True Class: Melanoma Skin Cancer Nevi and Moles, Predicted Class: Melanoma Skin Cancer Nevi and Moles
True Class: Melanoma Skin Cancer Nevi and Moles, Predicted Class: Melanoma Skin Cancer Nevi and Moles
True Class: Melanoma Skin Cancer Nevi and Moles, Predicted Class: Psoriasis pictures Lichen Planus and related diseases
True Class: Melanoma Skin Cancer Nevi and Moles, Predicted Class: Melanoma Skin Cancer Nevi and Moles
True Class: Melanoma Skin Cancer Nevi and Moles, Predicted Class: Melanoma Skin Cancer Nevi and Moles
True Class: Melanoma Skin Cancer Nevi and Mole